<a href="https://colab.research.google.com/github/CocoPig1028/ML_Note/blob/main/hyperParameterSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **검증세트**

In [74]:
# 훈련세트로 훈련하고 테스트세트로 검증한다.
# 계속 개선을 하게 되면 테스트세트에 잘맞는 모델이 만들어지는거 아닌가?
# 그래서 검증세트로 검증하고 테스트세트는 마지막 한번에서만 쓰는 방법이 있다.
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [75]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [76]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)
# 데이터를 훈련세트와 테스트세트로 분리

In [77]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)
# 훈련세트를 다시 훈련세트, 검증 세트로 분리

In [78]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [79]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


# **교차검증**

In [80]:
# 검증 세트를 만들었기 때문에 훈련세트가 줄었음
# 많은 데이터 -> 좋은 모델
# 교차 검증을 통해 안정적인 검증 점수 획득, 훈련에 더 많은 데이터를 사용
# k-fold cross validation
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)
# 5-폴드 교차 검증을 수행하였음
# 5개씩 해서 딕셔너리 출력
# 순서대로 훈련시간, 검증시간, 점수

{'fit_time': array([0.01228189, 0.01151752, 0.01286173, 0.01142001, 0.01186967]), 'score_time': array([0.00187445, 0.00183749, 0.00164509, 0.0013845 , 0.00214362]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [81]:
import numpy as np

print(np.mean(scores['test_score']))
# 검증 폴드의 점수 평균

0.855300214703487


In [82]:
# 교차 검증을 수행하면 입력한 모델에서 얻을 수 있는 최상의 검증 점수를 가늠 가능
# 다만 cross_validate()는 훈련 세트를 섞어 폴드를 나누지 않는다.
# 위에서 train_test_split()으로 전체데이터를 섞은 후 훈련
# 하지만 만약 교차 검증을 할 때 훈련 세트를 섞으려면 분할기를 지정
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))
# KFold 분할기를 사용하고 분류 모델일 경우 타깃 클래스를 골고루 나누기 위해 StratifiedKFold를 사용

0.855300214703487


In [83]:
# 훈련세트를 섞은 후 10-폴드 교차 검증을 수행하기
splitter =  StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


# **그리드 서치**

In [84]:
# 그리드 서치
# 결정 트리의 매개변수 값을 바꿔가며 성능이 좋은 모델 찾기
# 이때 테스트 세트를 사용하지 않고 교차 검증을 통해서 좋은 모델을 고르면 된다.
# 한 매개변수의 최적값을 찾고 다른 매개변수의 최적값을 찾는다 해도 둘이 섞으면 달라진다.
# 그래서 둘의 최적의 조합을 찾아야하는 것이 관건! -> 그리드 서치
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
# GridSearchCV를 이용해 하이퍼파라미터 탐색과 교차 검증을 한번에 수행
# GridSearchCV를 임포트하고 탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 만들기
# 여기서는 0.0001 부터 0.0005까지 0.0001씩 증가하는 5개의 값을 시도

In [85]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
# min_inpurity_decrease의 값을 바꿔가며 총 5번 실행
# 5-폴드 교차 검증을 5번씩 하기 때문에 25개의 모델을 훈련
# n_jobs 매개변수는 병렬 실행에 사용할 CPU 코어 수를 지정
# -1이면 전체 사용

In [86]:
gs.fit(train_input, train_target)
# 교차 검증에서 최적의 하이퍼파라민터를 찾으면 전체 훈련 세트로 모델을 다시 만들어야함
# 사이킷런의 그리드 서치는 훈련이 끝나면 25개의 모델 중에서 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델을 훈련
# 이 모델은 gs 객체의 best_estimator_ 속성에 저장

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [87]:
# 이 모델을 일반 결정 트리처럼 똑같이 사용 가능
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [88]:
# 그리드 서치로 찾은 최적의 매개변수는 best_params_ 속성에 저장
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [89]:
# 각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_result_ 속성의 'mean_test_score' 키에 저장
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [90]:
from pandas.compat.pyarrow import pa
# 수동으로 고르는 것보다 넘파이의 argmax() 함수를 사용하면 가장 큰 값의 인덱스 추출 가능
# 이후 이 인덱스를 이용하여 params 키에 저장된 매개변수를 출력 가능
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


1. 먼저 탐색할 매개변수를 지정
2. 그다음 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾는다. 그리고 이 조합은 그리드 서치 객체에 저장된다.
3. 그리드 서치는 최상의 매개변수에서 (교차 검증에 사용한 훈련 세트가 아니라)전체 훈련 세트를 사용해 최종 모델을 훈련한다. 이 모델도 그리드 서치 객체에 저장된다.

In [93]:
# 더 복잡합 매개변수 세트 찾기
# 노드를 분할하기 위한 불순도 감소 최소량을 지정
# max_depth로 트리의 깊이를 제한
# min_samples_split으로 노드를 나누기 위한 최소 샘플 수 선정
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)}
# 불순도 감소 최소량: 9개 세트
# 트리 깊이: 15개 세트
# 최소 샘플 수: 10개 세트
# 만들어지는 모델의 수: 9 * 15 * 10 = 6750개

In [94]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [95]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [98]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


# **랜덤서치**

In [99]:
# 위에서 탐색할 매개변수의 값을 0.0001 혹은 1로 설정
# 이렇게 간격을 둔 특별한 근거가 없음
# 이보다 더 좁거나 넓은 간격으로 시도해 볼 수 있을까?
# 랜덤 서치는 매개변수 값의 목록을 전달하는 것이 아님
# 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달
# scipy: 핵심 과학 라이브러리, 적분, 보간, 선형 대수, 확률 등을 포함한 수치 계산 전용 라이브러리
from scipy.stats import uniform, randint
# uniform과 randint 클래스 모두 주어진 범위에서 고르게 값을 추출
# 균등 분포에서 샘플링 한다.

In [101]:
# randint를 이용하여 숫자 10개를 샘플링
rgen = randint(0, 10)
rgen.rvs(10)

array([9, 7, 3, 1, 8, 0, 9, 3, 4, 1])

In [103]:
# 1000개를 샘플링
np.unique(rgen.rvs(1000), return_counts=True)
# 어느 정도 고르게 추출

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([106,  78,  95,  89, 101, 117, 106, 100,  98, 110]))

In [104]:
# 실수 10개 추출
ugen = uniform(0,1)
ugen.rvs(10)

array([0.29355843, 0.50810731, 0.07454642, 0.23520427, 0.55816094,
       0.5348729 , 0.7113139 , 0.90423746, 0.61540539, 0.80753672])

In [105]:
# min_samples_leaf 매개변수를 탐색 대상에 추가
# 리프 노드가 되기 위한 최소 샘플의 개수
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [106]:
from sklearn.model_selection import RandomizedSearchCV
# 사이킷런의 랜덤 서치 클래스
# n_iter 매개변수를 통해 샘플링 횟수 지정
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fbe265d3af0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fbe265d2aa0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fbe265d37c0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fbe265d3550>},
                   random_state=42)

In [107]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [109]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [111]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
